In [1]:
from jdftx import *

/home/main/anaconda2/envs/python3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/main/anaconda2/envs/python3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/main/anaconda2/envs/python3/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [36]:
o_0 = np.fromfile('data/W15OH1-confining.fluidN_H2O_O')
h_0 = np.fromfile('data/W15OH1-confining.fluidN_H2O_H')
n_0 = np.fromfile('data/W15OH1-confining.n')
o_0 = o_0.reshape(140,140,140)
h_0 = h_0.reshape(140,140,140)
n_0 = n_0.reshape(140,140,140)


In [37]:
a = 35*bohr_as_angstrom
dx = a/140
x = np.arange(0, a, dx); x+=a/2; x %= a; x-= a/2
y = x[:]; z = x[:]
from itertools import product
xyz = np.asarray(list(product(x,y,z)))

In [38]:
from IPython.html.widgets import *
from IPython import display
from chemview import enable_notebook, RepresentationViewer, MolecularViewer
enable_notebook()
%cd -q data
tv = jdlat().vis()
# !grep include W15OH1-confining.in
%cd -q ../

N = 10000
randNPoints = np.random.choice(xyz.shape[0], N, replace=False)
peakPointsO = (o_0>6*o_0.std()+o_0.mean()).reshape(-1)
peakPointsH = (h_0>6*h_0.std()+h_0.mean()).reshape(-1)
peakPointsn = (n_0>2*n_0.std()+n_0.mean()).reshape(-1)

indChoiceO = peakPointsO
N_O = np.size(xyz[indChoiceO])

oxygen = tv.add_representation('points', {'coordinates' : xyz[indChoiceO], 
                                 'sizes' : o_0.reshape(-1)[indChoiceO]*10, 
                                 'colors' : [0xAA0000]*N_O })
oSize = 10

indChoiceH = peakPointsH
N_H = np.size(xyz[indChoiceH])

hSize = 10
hydrogen = tv.add_representation('points', {'coordinates' : xyz[indChoiceH], 
                                 'sizes' : h_0.reshape(-1)[indChoiceH]*hSize, 
                                 'colors' : [0xAAAAAA]*N_H })

nSize = 1
indChoice_n = peakPointsn
N_n = np.size(xyz[indChoice_n])

n = tv.add_representation('points', {'coordinates' : xyz[indChoice_n], 
                                 'sizes' : n_0.reshape(-1)[indChoice_n], 
                                 'colors' : [0x00AA00]*N_n })
print(n)
print(len(tv.representations))
def update_n(tv, n, size, lowerLim, upperLim):
    ind_n = np.logical_and(n_0>lowerLim, n_0<upperLim).reshape(-1)
    if not ind_n.any():
        return n
    if np.sum(ind_n)>10000:
        print("Too many points")
        return n
    else:
        print(np.sum(ind_n))
    tv.remove_representation(n)
    n = tv.add_representation('points', {'coordinates' : xyz[ind_n], 
                                         'sizes' : n_0.reshape(-1)[ind_n]*size, 
                                         'colors' : [0x00AA00]*len(xyz[ind_n]) })
    return n

@interact(nSize=(0.1,13,.1), 
          n_lowerLim=(0, 1, 0.001),
          n_upperLim=(0.001, 1, 0.001)
          )
def change_n_Size(nSize=0.1, n_lowerLim=0.1, n_upperLim=1):#, ln_oxy_size, ln_h_size, H_threshold):
    global n
    nSize = np.exp(nSize)
    n = update_n(tv, n, nSize, n_lowerLim, n_upperLim)
    
@interact(O_threshold=(o_0.mean(), o_0.mean()+o_0.std()*10, o_0.std()))
def changeOxygenThreshold(O_threshold):
    global oxygen, oSize
    peakPointsO = (o_0>O_threshold).reshape(-1)
    N_O = np.size(xyz[peakPointsO])
    tv.remove_representation(oxygen)
    oxygen = tv.add_representation('points', {'coordinates' : xyz[peakPointsO], 
                                 'sizes' : o_0.reshape(-1)[peakPointsO]*oSize, 
                                 'colors' : [0xAA0000]*N_O })
    
@interact(H_threshold=(h_0.mean(), h_0.mean()+h_0.std()*10, h_0.std()))
def changeHydrogenThreshold(H_threshold):
    global hydrogen, hSize
    peakPointsH = (h_0>H_threshold).reshape(-1)
    N_H = np.size(xyz[peakPointsH])
    tv.remove_representation(hydrogen)
    hydrogen = tv.add_representation('points', {'coordinates' : xyz[peakPointsH], 
                                     'sizes' : h_0.reshape(-1)[peakPointsH]*hSize, 
                                     'colors' : [0xAAAAAA]*N_H })

display.display(tv)

In [33]:
display.display(tv)

In [39]:
from IPython.html.services.config import ConfigManager
from IPython.display import HTML
ip = get_ipython()
cm = ConfigManager(parent=ip, profile_dir=ip.profile_dir.location)
extensions =cm.get('notebook')
table = ""
for ext in extensions['load_extensions']:
    table += "<tr><td>%s</td>\n" % (ext)

top = """
<table border="1">
  <tr>
    <th>Extension name</th>
  </tr>
"""
bottom = """
</table>
"""
HTML(top + table + bottom)

Extension name
chemview
